In [490]:
import tools
import numpy as np
import scipy as sc
import sympy as sp
import matplotlib.pyplot as plt
import IPython.display as disp

In [491]:
# My symbol Class

class MySymbol(sp.Symbol):
    _eq_n = 0

    @classmethod
    def set_eq_n(self, value):
        self._eq_n = value

    def get_eq_n(self):
        return self._eq_n

    eq_n = property(get_eq_n, set_eq_n)

    def __new__(cls, name, expr=None):
        obj = sp.Symbol.__new__(cls, name)
        obj.name = name
        if isinstance(expr,str):
            obj.expr = sp.Symbol.__new__(cls, expr)
        elif expr is None:
            obj.expr = obj
        else:
            obj.expr = expr
        return obj
    
    def subs(self, *args, **kwargs):
        new_expr = self.expr.subs(*args, **kwargs)
        return MySymbol(self.name,new_expr)
    
    def get_expr(self,*args,simplify=False):
        expr = self.expr
        for arg in args:
            if isinstance(arg,MySymbol):
                expr = expr.subs(arg,arg.expr)
        if simplify:
            expr = sp.simplify(expr)
        return expr

    def get_expr_all(self,*args,simplify=False):
        expr = self.expr
        symbols = list(self.expr.free_symbols)
        for symbol in symbols:
            if isinstance(symbol,MySymbol) and symbol not in args:
                expr = expr.subs(symbol,symbol.get_expr_all(*args))
        if simplify:
            expr = sp.simplify(expr)
        return expr
    
    def print_expr(self,*args,simplify=False,pre_print=''):
        expr = self.get_expr(*args,simplify)
        self.display(expr,pre_print)

    
    def print_expr_all(self,*args,simplify=False,pre_print=''):
        expr = self.get_expr_all(*args,simplify)
        self.display(expr,pre_print)
        
    def display(self,expr,pre_print=''):
        if pre_print != '':
            print(f'({self.eq_n}{pre_print}):')
        else:
            self.set_eq_n(self.get_eq_n()+1)
            print(f'({self.eq_n}):')
        disp.display(sp.Eq(self,expr))
        print()


# Useful functions
def parallel_impedance(*arg):
    impedances = np.asarray(arg)
    parallel_impedance = 1/np.sum(1/impedances)
    return sp.simplify(parallel_impedance)

def get_omega(f):
    return 2*sp.pi*f

# AC Symbols
f = sp.Symbol('f')
omega = MySymbol('\\omega',get_omega(f))
s = MySymbol('s',sp.I*omega)

    

# Project noteboook

See complementary information in the Project manuscript.

In [492]:
# Component Symbols
R1 = sp.Symbol('R1', real=True)
C1 = sp.Symbol('C1', real=True)
C1s = 1/(s*C1)


R2 = sp.Symbol('R2', real=True)
C2 = sp.Symbol('C2', real=True)
C2s = 1/(s*C2)

RG = sp.Symbol('R_G', real=True)

i_inp = sp.Symbol('i_in,p')
i_inm = sp.Symbol('i_in,m')

V_ref = sp.Symbol('V_ref')
V_out = MySymbol('V_out')

## Circuit Analysis

In [493]:
# OpAmp non-idealities
def A(s,A_0,omega_c):
    return A_0/(1+s/omega_c)

class OpAmp:
    def __init__(self,name):
        self.name = name
        self.A_0 = sp.Symbol(f'A_0,{self.name}')
        self.omega_c = sp.Symbol(f'\\omega_c,{self.name}')
        self.A = MySymbol(f'A_{self.name}',A(s,self.A_0,self.omega_c))

        self.s_n_v_in_white = MySymbol(f'S_{{n,v_{{in}},white,{self.name}}}')
        self.s_n_i_in_white = MySymbol(f'S_{{n,i_{{in}},white,{self.name}}}')

        self.f_c_n_v_in = sp.Symbol(f'f_{{c,n,v_{{in}},{self.name}}}')
        self.f_c_n_i_in = sp.Symbol(f'f_{{c,n,i_{{in}},{self.name}}}')

        self.s_n_v_in_1_omega_white = None
        self.s_n_i_in_1_omega_white = None

        

### C1

In [494]:
i_in = sp.Symbol('i_in')
V_out_1 = MySymbol('V_out_1')

Vm = MySymbol('V_m',V_out_1 - i_in * parallel_impedance(R1,C1s))
Vp = MySymbol('V_p',V_ref * (C2s/(R2+C2s)))
Vm.print_expr()
Vp.print_expr()

OP1 = OpAmp('OP1')
OP1.A.print_expr()

V_out_1.expr = (Vp-Vm)*OP1.A
V_out_1.print_expr()
V_out_1.expr = sp.simplify(sp.collect(sp.collect(sp.factor(sp.solve(V_out_1-V_out_1.get_expr(Vm,Vp),V_out_1)[0]),i_in),V_ref))
V_out_1.print_expr()

V_out_1_prime = V_out_1.subs(V_ref,0)
V_out_1_prime.print_expr(pre_print="'")

H_1 = MySymbol('H_{1}(s)', V_out_1.expr/i_in)
H_1.print_expr(pre_print="'*")


(1):


Eq(V_m, V_out_1 - R1*i_in/(s*C1*R1 + 1))


(2):


Eq(V_p, V_ref/(s*C2*(R2 + 1/(s*C2))))


(3):


...

Eq(V_out_1, A_OP1*(R1*i_in*(s*C2*R2 + 1) + V_ref*(s*C1*R1 + 1))/((A_OP1 + 1)*(s*C1*R1 + 1)*(s*C2*R2 + 1)))


(5'):


Eq(V_out_1, A_OP1*R1*i_in/((A_OP1 + 1)*(s*C1*R1 + 1)))


(5'*):


Eq(H_{1}(s), A_OP1*R1/((A_OP1 + 1)*(s*C1*R1 + 1)))

### C2

In [495]:
V_in_p = MySymbol('V_{in,p}')
V_in_m = MySymbol('V_{in,m}')
V_out = MySymbol('V_out')

# AD8429
def Gain(RG):
    return 1+ 6e3/RG

G = MySymbol('G(R_{G})',Gain(RG))
G.print_expr()

OP2 = OpAmp('OP2')
OP2.A.print_expr()

dV_in = MySymbol('\\Delta V_in',V_in_p-V_in_m)
dV_in.print_expr()

V_out.expr = (V_in_p-V_in_m)*OP2.A*G
V_out.expr = V_out.expr.subs(dV_in.expr,dV_in)
V_out.print_expr()

H_2 = MySymbol('H_{2}(s)', V_out.expr/dV_in)
H_2.print_expr(pre_print="'*")

(6):


Eq(G(R_{G}), 1 + 6000.0/R_G)


(7):


Eq(A_OP2, A_0,OP2/(s/\omega_c,OP2 + 1))


(8):


Eq(\Delta V_in, -V_{in,m} + V_{in,p})


(9):


Eq(V_out, A_OP2*G(R_{G})*\Delta V_in)


(9'*):


Eq(H_{2}(s), A_OP2*G(R_{G}))

### Ctot

In [496]:
V_out_1_p = MySymbol('V_out1,p','V_{out1}(i_{inp})')
V_out_1_m = MySymbol('V_out1,m','V_{out1}(i_{inm})')
V_out_1_p.print_expr()
V_out_1_m.print_expr(pre_print="'")

V_in_p.expr = V_out_1_p
V_in_m.expr = V_out_1_m
V_in_p.print_expr()
V_in_m.print_expr(pre_print="'")

dV_in.expr = sp.simplify(V_out_1.expr.subs(i_in,i_inp) - V_out_1.expr.subs(i_in,i_inm))
dV_in.print_expr()

di_in = MySymbol('\\Delta i_in',i_inp-i_inm)
di_in.print_expr()

V_out.expr = sp.simplify(V_out.expr.subs(dV_in,dV_in.expr))
V_out.expr = V_out.expr.subs(di_in.expr,di_in)
V_out.print_expr()

H_tot = MySymbol('H_{tot}(s)', V_out.expr/di_in)
H_tot.print_expr(pre_print="'*")

(10):


Eq(V_out1,p, V_{out1}(i_{inp}))


(10'):


Eq(V_out1,m, V_{out1}(i_{inm}))


(11):


...

Eq(\Delta i_in, -i_in,m + i_in,p)


(14):


Eq(V_out, A_OP1*A_OP2*G(R_{G})*\Delta i_in*R1/((A_OP1 + 1)*(s*C1*R1 + 1)))


(14'*):


Eq(H_{tot}(s), A_OP1*A_OP2*G(R_{G})*R1/((A_OP1 + 1)*(s*C1*R1 + 1)))

## Noise Analysis

In [497]:
k = sp.Symbol('k')
T = sp.Symbol('T')

def n_thermal_v_sd(Z):
    R = sp.re(Z)
    if R == float('inf'):
        return 0
    return 4*k*T*R

def new_thermal_v_sd(Z):
    return MySymbol(f'S_{{TN({Z.name}),v}}', n_thermal_v_sd(Z))

In [498]:
def get_s_n_1_f(f,f_c,white_noise):
    C = sp.Symbol('C')
    n_1_f_sd = sp.solve(C/f_c - white_noise,C)[0]/f
    return n_1_f_sd

def get_s_n_1_omega(omega,f_c,white_noise):
    return get_s_n_1_f(omega,get_omega(f_c),white_noise)

def get_s_n_1_omega_white(omega,f_c,white_noise):
    return get_s_n_1_omega(omega,f_c,white_noise) + white_noise

def set_s_n_v_in_1_omega_white(self):
    self.s_n_v_in_1_omega_white = MySymbol(f'S_{{n,v_{{in}},{self.name}}}(\\omega)',get_s_n_1_omega_white(omega,self.f_c_n_v_in,self.s_n_v_in_white))
setattr(OpAmp, 'set_s_n_v_in_1_omega_white', set_s_n_v_in_1_omega_white)

def set_s_n_i_in_1_omega_white(self):
    self.s_n_i_in_1_omega_white = MySymbol(f'S_{{n,i_{{in}},{self.name}}}(\\omega)',get_s_n_1_omega_white(omega,self.f_c_n_i_in,self.s_n_i_in_white))
setattr(OpAmp, 'set_s_n_i_in_1_omega_white', set_s_n_i_in_1_omega_white)

OP1.set_s_n_v_in_1_omega_white()
OP1.set_s_n_i_in_1_omega_white()
OP2.set_s_n_v_in_1_omega_white()
OP2.set_s_n_i_in_1_omega_white()

OP1.s_n_v_in_1_omega_white.print_expr()
OP1.s_n_i_in_1_omega_white.print_expr(pre_print="'")

(15):


Eq(S_{n,v_{in},OP1}(\omega), S_{n,v_{in},white,OP1} + 2*S_{n,v_{in},white,OP1}*pi*f_{c,n,v_{in},OP1}/\omega)


(15'):


Eq(S_{n,i_{in},OP1}(\omega), S_{n,i_{in},white,OP1} + 2*S_{n,i_{in},white,OP1}*pi*f_{c,n,i_{in},OP1}/\omega)

In [499]:
OP2.s_n_v_out_white = sp.Symbol(f'S_{{n,v_{{out}},white,{OP2.name}}}')

In [500]:
noise_dict_template = {'Noise Spectral Density': None,
                       'Transfer Function': None}

def new_noise(sn,H_expr):
    H = MySymbol(sn.name.replace('S_','H_').removesuffix('(s)')+'(s)',H_expr)
    new_noise_dict = noise_dict_template.copy()
    new_noise_dict['Noise Spectral Density'] = sn
    new_noise_dict['Transfer Function'] = H
    return new_noise_dict

def add_noise(sn,H_expr,noise_list):
    noise_list.append(new_noise(sn,H_expr))

### Cn1

In [501]:
cn1_list = []
def add_cn1_noise(sn,H_expr):
    add_noise(sn,H_expr,cn1_list)

Z_in = MySymbol('Z_{in}',parallel_impedance(R1,C1s))
Z_ref = MySymbol('Z_{ref}',parallel_impedance(R2,C2s))

add_cn1_noise(new_thermal_v_sd(Z_ref),OP1.A/(1+OP1.A))
add_cn1_noise(OP1.s_n_v_in_1_omega_white,OP1.A/(1+OP1.A))
add_cn1_noise(OP1.s_n_i_in_1_omega_white,Z_ref*OP1.A/(1+OP1.A))
add_cn1_noise(OP1.s_n_i_in_1_omega_white,-Z_in*OP1.A/(1+OP1.A))
add_cn1_noise(new_thermal_v_sd(Z_in),1)
add_cn1_noise(OP2.s_n_i_in_1_omega_white,-Z_in*OP1.A/(1+OP1.A))

for noise in cn1_list:
    for name,symb in noise.items():
        print(name+':')
        symb.print_expr()
    print()

Noise Spectral Density:
(16):


Eq(S_{TN(Z_{ref}),v}, 4*T*k*re(Z_{ref}))


Transfer Function:
(17):


Eq(H_{TN(Z_{ref}),v}(s), A_OP1/(A_OP1 + 1))



Noise Spectral Density:
(18):


...

Eq(H_{TN(Z_{in}),v}(s), 1)



Noise Spectral Density:
(26):


Eq(S_{n,i_{in},OP2}(\omega), S_{n,i_{in},white,OP2} + 2*S_{n,i_{in},white,OP2}*pi*f_{c,n,i_{in},OP2}/\omega)


Transfer Function:
(27):


Eq(H_{n,i_{in},OP2}(\omega)(s), -A_OP1*Z_{in}/(A_OP1 + 1))